# Numerical derivatives 

In this section we will learn how we can calculate a derivative numerically. The fundamental idea involves replacing differentials with differences. An important aspect is to think about the types of errors that emerge when doing that, especially we have to introduce the important concepts of rounding error and truncation errors.

**Literature:** Numerical Recipies, Ch. 5.7

## Difference equations
Function
$$ y = f(x) $$ for example $$y = x^2$$ for $x \in [1,5]$.

In [ ]:
%pylab nbagg 
x=linspace(5,25,25)

In [ ]:
figure(1)
xl =  4; xr = 22
f = lambda x: x**2    # function
x0 = 15               # take derivative at x0
ff = lambda x: 2*x0*x + f(x0) -2*x0**2   # derivative
close(1);figure(1)
plot(x,f(x)); xlabel('x'), ylabel('f(x)'); ylim(0,500)
if True:
    plot(x0,f(x0),'o')
    plot(x[4:22],ff(x[4:22]),'-.',lw=2)
if True:
    x1=x[6];x2=x[18]
    plot([x1,x2],[ff(x1),ff(x1)],'k--x',lw=1)
    plot([x2,x2],[ff(x1),ff(x2)],'k--x',lw=1)
    text(0.5*(x2+x1),1.1*ff(x1),'$ \Delta x$')
    text(1.02*x2,0.5*(ff(x1)+ff(x2)),'$ \Delta y$')
    text(x1,0.5*ff(x1),'$x_1$')
    text(x2,0.5*ff(x1),'$x_2$')
    text(1.02*x2,ff(x1),'$y_1$')
    text(1.02*x2,0.9*ff(x2),'$y_2 = ff(x_2)$')

### Slope
$$
\frac{dy}{dx} = \frac{\Delta y}{\Delta x} = 
\frac{y_2 - y_1}{x_2 - x_1}
$$

## Derivative 
The derivative $\frac{df}{dx}$ of a function $y=f(x)$ can be approximated by the difference equation 
$$ f'(x) \approx \frac{f(x+h) -f(x)}{h}.$$

Why? Rearrange the Taylor expansion of $f(x)$
$$
f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$
to solve for $f^\prime(x)$ and discard order two and higher terms
$$
\frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$

In [ ]:
def deriv1(f,x,h):
    dfdx = (f(x+h) - f(x)) / h
    return dfdx

In [ ]:
deriv1(f,1,1.e-3) - 2. 

Note that the answer is not exactly `2.00000000` as we know the answer to be from the analytical approach. But smaller values of `h` give a better answer? _Right?_ well, let's see .... let's analyse the error in a bit more detail. The higher order terms that we discarded represent the truncation error. They represent the difference between the solution of the difference equation and the exact mathematical equation, no matter how _precisely_ the difference equation is solved. The truncation error limits the _accuracy_ of the method. Note the difference between _precision_ and _accuracy_ introducecd here!

We can improve the accuracy by taking smaller $h$ for calculating the derivative. Can't we make the answer then arbitrarily exact? Let's try ...

## Errors
There are two basic sources of error! They have to do with accuracy and with precision.

**Accuracy:** How well does the discretized equation represent the original mathematical equation?
This is the truncation error. 

**Precision:** How well is the solution satisfying the discretized equation? This is limited by the roundoff error.

### Convergence test
In order to determine the accuracy of our solution scheme we study the behaviour of a particular solution scheme under grid (time and/or space) refinement. In our case this means we do a series of runs with decreasing `h`.

We use a convenient technique for _vectorizing_ a function which allows to evaluate an arbitrary function for a _parameter vector_. This way of doing it will not help performance. See an example below of how to do true multi-tasking.

In [ ]:
# in order to vectorize wrap the function to isolate the 
# variable over which we should loop turn a scalar function 
# into a vectorized function
def hdev(h):
    return deriv1(f,1.,h)
vhdev = vectorize(hdev)
h_pow = range(0,-14,-1)

In [ ]:
h = 10**array(h_pow, dtype=float)
figure(2,figsize=(8,8))
plot(h_pow, log10(vhdev(h)-2.0),'o')
xlim(0,-14), ylabel('log10 (df/dx - 2.0)'), xlabel('log h')

You can reduce the truncation error by taking a smaller interval `h`. However, as you try minimizing the truncation error you will see that there is a limit. At too small `h` the `deriv1` function will try to perform the difference `(f(x+h) - f(x))` with too few or no significant digits and the roundoff error will become larger, and eventually dominate the result. 


## Multi-processing

We saw an example for a case where a particular task needs to be done over and over again with different input parameters. A previous example of this type was the Monte Carlo method. Such problems are called _embarissingly parallel_, essentially because it takes no or very little effort to parallelize them. The following introduces an actual multiprocessing approach that will use multiple of your cpu cores. The work is too small to notice any difference in performance though. You may want to experiment with a task that has more work, and in an environment that has multiple cores. 

In [ ]:
from multiprocessing import Pool

In [ ]:
p = Pool(3)
h_error = array(p.map(hdev,h))
print(h_error)
figure(3)
plot(h_pow,log10(abs(h_error-2.)),'o')
xlim(0,-16), ylabel('$ \log$ $df/dx - 2.0 $'),xlabel('$\log h$')

# Ordinary differential equations

Differential equations are often hard to solve on paper but in many cases become trivial on a computer. 

## Euler step
Take the simplest, first order ODE
$$
y^\prime = f(y,x)
$$
where the right-hand side (RHS) is the function $f(y,x)$ that specifies the derivative $y^\prime = \frac{dy}{dx}$. We are looking for the function $y(x)$, but here not the algebraic expression but the numerical values. For a time dependent problem $x = t$. 

Take for example $f (y,x) = 2x$, then we know that $y(x) = x^2$. Let's pretend we do not know the answer, but the initial conditions $y(0) = 0$. How can we numerically calculate $y(x)$ for a series of discrete values $x_i$?

In [ ]:
y=[]; y.append(0)
x=[]; x.append(0)
rhs_f = lambda x: 2*x
x_thing = x[0]; y_thing = y[0]
dx=1.; x_end = 4.
while x_thing <= x_end+dx:
    y_thing += dx * rhs_f(x_thing)
    x_thing += dx
    print(x_thing,y_thing)
    x.append(x_thing); y.append(y_thing)


In [ ]:
figure(4)
plot(x,y,'o-',label='numeric')
plot(x,array(x)**2,'--',label='analytic')
legend();xlabel('$x$'),ylabel('$y(x)$')